# Working with CategoricalArrays

In [ ]:
using DataFrames # load package
using CategoricalArrays # CategoricalArrays.jl is independent from DataFrames.jl but it is often used in combination

## Constructor

In [ ]:
x = categorical(["A", "B", "B", "C"]) # unordered

In [ ]:
y = categorical(["A", "B", "B", "C"], ordered=true) # ordered, by default order is sorting order

In [ ]:
z = categorical(["A","B","B","C", missing]) # unordered with missings

In [ ]:
c = cut(1:10, 5) # ordered, into equal counts, possible to rename labels and give custom breaks

(we will cover grouping later, but let us here use it to analyze the results, we use Chain.jl for chaining)

In [ ]:
using Chain

In [ ]:
@chain DataFrame(x=cut(randn(100000), 10)) begin
      groupby(:x)
      combine(nrow) # just to make sure cut works right
end

In [ ]:
v = categorical([1,2,2,3,3]) # contains integers not strings

In [ ]:
Vector{Union{String, Missing}}(z) # sometimes you need to convert back to a standard vector

## Managing levels

In [ ]:
arr = [x,y,z,c,v]

In [ ]:
isordered.(arr) # chcek if categorical array is orderd

In [ ]:
ordered!(x, true), isordered(x) # make x ordered

In [ ]:
ordered!(x, false), isordered(x) # and unordered again

In [ ]:
levels.(arr) # list levels

In [ ]:
unique.(arr) # missing will be included

In [ ]:
y[1] < y[2] # can compare as y is ordered

In [ ]:
v[1] < v[2] # not comparable, v is unordered although it contains integers

In [ ]:
y[2] < "A" # comparison against type underlying categorical value is not allowed

In [ ]:
y[2] < CategoricalValue("A", y) # you need to explicitly convert a value to a level

In [ ]:
y[2] < CategoricalValue("Z", y) # but it is treated as a level, and thus only valid levels are allowed

In [ ]:
levels!(y, ["C", "B", "A"]) # you can reorder levels, mostly useful for ordered CategoricalArrays

In [ ]:
y[1] < y[2] # observe that the order is changed

In [ ]:
levels!(z, ["A", "B"]) # you have to specify all levels that are present

In [ ]:
levels!(z, ["A", "B"], allowmissing=true) # unless the underlying array allows for missings and force removal of levels

In [ ]:
z[1] = "B"
z # now z has only "B" entries

In [ ]:
levels(z) # but it remembers the levels it had (the reason is mostly performance)

In [ ]:
droplevels!(z) # this way we can clean it up
levels(z)

## Data manipulation

In [ ]:
x, levels(x)

In [ ]:
x[2] = "0"
x, levels(x) # new level added at the end (works only for unordered)

In [ ]:
v, levels(v)

In [ ]:
v[1] + v[2] # even though the underlying data is Int, we cannot operate on it

In [ ]:
Vector{Int}(v) # you have either to retrieve the data by conversion (may be expensive)

In [ ]:
unwrap(v[1]) + unwrap(v[2]) # or get a single value

In [ ]:
unwrap.(v) # this will work for arrays witout missings

In [ ]:
unwrap.(z) # also works on missing values

In [ ]:
Vector{Union{String, Missing}}(z) # or do the conversion

In [ ]:
recode([1,2,3,4,5,missing], 1=>10) # recode some values in an array; has also in place recode! equivalent

In [ ]:
recode([1,2,3,4,5,missing], "a", 1=>10, 2=>20) # here we provided a default value for not mapped recodings

In [ ]:
recode([1,2,3,4,5,missing], 1=>10, missing=>"missing") # to recode Missing you have to do it explicitly

In [ ]:
t = categorical([1:5; missing])
t, levels(t)

In [ ]:
recode!(t, [1,3]=>2)
t, levels(t) # note that the levels are dropped after recode

In [ ]:
t = categorical([1,2,3], ordered=true)
levels(recode(t, 2=>0, 1=>-1)) # and if you introduce a new levels they are added at the end in the order of appearance

In [ ]:
t = categorical([1,2,3,4,5], ordered=true) # when using default it becomes the last level
levels(recode(t, 300, [1,2]=>100, 3=>200))

## Comparisons

In [ ]:
x = categorical([1,2,3])
xs = [x, categorical(x), categorical(x, ordered=true), categorical(x, ordered=true)]
levels!(xs[2], [3,2,1])
levels!(xs[4], [2,3,1])
[a == b for a in xs, b in xs] # all are equal - comparison only by contents

In [ ]:
signature(x::CategoricalArray) = (x, levels(x), isordered(x)) # this is actually the full signature of CategoricalArray
# all are different, notice that x[1] and x[2] are unordered but have a different order of levels
[signature(a) == signature(b) for a in xs, b in xs]

In [ ]:
x[1] < x[2] # you cannot compare elements of unordered CategoricalArray

In [ ]:
t[1] < t[2] # but you can do it for an ordered one

In [ ]:
isless(x[1], x[2]) # isless works within the same CategoricalArray even if it is not ordered

In [ ]:
y = deepcopy(x) # but not across categorical arrays
isless(x[1], y[2])

In [ ]:
isless(unwrap(x[1]), unwrap(y[2])) # you can use get to make a comparison of the contents of CategoricalArray

In [ ]:
x[1] == y[2] # equality tests works OK across CategoricalArrays

## Categorical columns in a DataFrame

In [ ]:
df = DataFrame(x = 1:3, y = 'a':'c', z = ["a","b","c"])

Convert all `String` columns to categorical in-place

In [ ]:
transform!(df, names(df, String) => categorical, renamecols=false)

In [ ]:
describe(df)